# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [1]:
%pylab inline
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [2]:
import pandas as pd
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)
test = pd.read_csv('test.csv', index_col=0)

In [3]:
target2 = target

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [4]:
from sklearn.model_selection import train_test_split
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

In [5]:
data[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
783681,0,2,0,1,0,1,0,0,0,0,...,3,1,5,9,0,0,1,0,1,0
636497,0,1,6,1,0,0,1,0,0,0,...,5,2,1,10,0,1,1,0,1,0
659813,2,1,3,1,0,1,0,0,0,0,...,3,1,1,8,0,1,1,0,0,0
334742,0,1,10,0,0,1,0,0,0,0,...,7,2,2,11,0,0,1,1,0,0
1156570,0,1,9,0,6,1,0,0,0,0,...,8,1,1,8,0,1,0,0,1,0
1423305,0,2,1,0,0,1,0,0,0,0,...,8,3,1,8,0,0,0,1,1,1
1092591,1,2,1,1,3,0,0,0,1,0,...,7,3,2,3,0,1,1,0,1,1
655398,1,1,3,0,3,1,0,0,0,0,...,7,0,3,10,0,0,0,1,0,0
1363991,1,1,2,1,0,0,1,0,0,0,...,7,1,0,4,0,1,0,1,1,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 783681 to 1320118
Data columns (total 57 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ps_ind_01       200000 non-null  int64  
 1   ps_ind_02_cat   200000 non-null  int64  
 2   ps_ind_03       200000 non-null  int64  
 3   ps_ind_04_cat   200000 non-null  int64  
 4   ps_ind_05_cat   200000 non-null  int64  
 5   ps_ind_06_bin   200000 non-null  int64  
 6   ps_ind_07_bin   200000 non-null  int64  
 7   ps_ind_08_bin   200000 non-null  int64  
 8   ps_ind_09_bin   200000 non-null  int64  
 9   ps_ind_10_bin   200000 non-null  int64  
 10  ps_ind_11_bin   200000 non-null  int64  
 11  ps_ind_12_bin   200000 non-null  int64  
 12  ps_ind_13_bin   200000 non-null  int64  
 13  ps_ind_14       200000 non-null  int64  
 14  ps_ind_15       200000 non-null  int64  
 15  ps_ind_16_bin   200000 non-null  int64  
 16  ps_ind_17_bin   200000 non-null  int64  
 17  ps_ind_18

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [7]:
datatypes = data.dtypes
ind = 0
for i in data:
    if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
        mn = min(X_train[i].min(), X_test[i].min())
        mx = max(X_train[i].max(), X_test[i].max())
        X_train[i] = (X_train[i] - mn) / (mx - mn)
        X_test[i] = (X_test[i] - mn) / (mx - mn)
        test[i] = (test[i] - mn) / (mx - mn)
    ind += 1

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1106723606.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1106723606.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1106723606.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as 

In [8]:
X_train[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
1049183,0.000000,0.4,0.454545,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.222222,0.375,0.307692,0.272727,0.0,0.0,0.0,0.0,0.0,0.0
106872,0.142857,0.4,0.272727,1.0,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.500000,0.000,0.307692,0.545455,0.0,1.0,1.0,0.0,0.0,0.0
28943,0.857143,0.6,0.636364,1.0,0.142857,0.0,0.0,1.0,0.0,0.0,...,0.222222,0.125,0.153846,0.318182,0.0,1.0,1.0,0.0,1.0,0.0
851826,0.142857,0.4,0.454545,1.0,0.142857,0.0,0.0,1.0,0.0,0.0,...,0.333333,0.250,0.153846,0.454545,0.0,0.0,1.0,0.0,0.0,0.0
934813,0.000000,0.4,0.090909,0.5,0.714286,1.0,0.0,0.0,0.0,0.0,...,0.500000,0.125,0.153846,0.636364,0.0,1.0,1.0,0.0,0.0,0.0
1147039,0.285714,0.4,0.181818,1.0,0.142857,0.0,0.0,1.0,0.0,0.0,...,0.222222,0.250,0.076923,0.545455,0.0,1.0,0.0,0.0,0.0,0.0
978813,0.000000,0.4,0.181818,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.111111,0.000,0.153846,0.272727,0.0,0.0,1.0,0.0,1.0,0.0
775504,0.428571,0.4,1.000000,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.166667,0.250,0.384615,0.409091,0.0,1.0,0.0,0.0,1.0,0.0
371031,0.000000,0.4,0.181818,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.333333,0.125,0.076923,0.590909,0.0,1.0,1.0,0.0,1.0,0.0


In [9]:
X_test[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
1050571,0.142857,0.6,0.181818,0.5,0.142857,0.0,0.0,1.0,0.0,0.0,...,0.388889,0.250,0.307692,0.318182,0.0,0.0,0.0,0.0,1.0,0.0
134934,0.571429,0.8,0.818182,1.0,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.388889,0.000,0.230769,0.227273,0.0,1.0,1.0,1.0,0.0,0.0
630943,0.000000,0.6,0.090909,0.5,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.222222,0.250,0.230769,0.363636,0.0,1.0,0.0,0.0,0.0,0.0
48209,1.000000,0.8,0.636364,0.5,0.142857,0.0,0.0,0.0,1.0,0.0,...,0.388889,0.125,0.153846,0.272727,1.0,0.0,1.0,0.0,1.0,0.0
867010,0.000000,0.4,0.363636,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.222222,0.125,0.230769,0.272727,0.0,1.0,1.0,1.0,0.0,0.0
1352938,0.000000,0.4,0.454545,1.0,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.222222,0.125,0.384615,0.409091,0.0,1.0,1.0,1.0,0.0,0.0
55384,1.000000,0.4,0.454545,0.5,0.714286,0.0,0.0,1.0,0.0,0.0,...,0.222222,0.250,0.153846,0.454545,0.0,1.0,1.0,0.0,1.0,0.0
1093377,0.000000,0.4,0.181818,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.166667,0.125,0.230769,0.136364,0.0,0.0,1.0,0.0,1.0,0.0
173007,1.000000,0.8,0.727273,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.333333,0.125,0.153846,0.454545,1.0,0.0,1.0,0.0,1.0,0.0


In [10]:
test[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.4,0.727273,1.0,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.055556,0.125,0.076923,0.545455,0.0,1.0,1.0,0.0,0.0,1.0
1,0.571429,0.6,0.454545,1.0,0.142857,0.0,0.0,0.0,1.0,0.0,...,0.111111,0.000,0.230769,0.454545,0.0,0.0,1.0,1.0,0.0,1.0
2,0.714286,0.4,0.272727,0.5,0.142857,0.0,0.0,0.0,1.0,0.0,...,0.222222,0.000,0.153846,0.181818,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.4,0.545455,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.277778,0.125,0.000000,0.227273,1.0,0.0,1.0,0.0,0.0,0.0
4,0.714286,0.4,0.636364,0.5,0.142857,0.0,0.0,0.0,1.0,0.0,...,0.222222,0.000,0.000000,0.181818,0.0,1.0,1.0,0.0,0.0,1.0
5,0.000000,0.4,0.545455,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.444444,0.125,0.307692,0.409091,1.0,0.0,1.0,0.0,1.0,0.0
6,0.000000,0.4,0.272727,0.5,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.111111,0.000,0.307692,0.272727,1.0,1.0,0.0,0.0,0.0,0.0
8,0.000000,0.4,0.000000,0.5,0.142857,1.0,0.0,0.0,0.0,0.0,...,0.166667,0.125,0.307692,0.409091,0.0,1.0,0.0,0.0,0.0,0.0
10,0.000000,0.4,0.636364,0.5,0.142857,0.0,1.0,0.0,0.0,0.0,...,0.277778,0.125,0.307692,0.272727,0.0,0.0,1.0,0.0,0.0,0.0


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

logistic_regression = LogisticRegression(C=1.0, penalty='l2', solver='lbfgs', random_state = 42)
start_time = time.time()
logistic_regression.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time)

# Making predictions on the test set
y_pred = logistic_regression.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculating precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculating recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculating F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Training time: 1.5472581386566162
Accuracy: 0.58827
Precision: 0.5957257732405662
Recall: 0.5496340439147303
F1 score: 0.5717524936812873


/Users/kseniashk/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


__Выводы__ в свободной форме:
мало, но очень быстро

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = []
ind = 0
for i in data:
    if i[-3:] == 'cat':
        categorical_features.append(ind)
    ind += 1
print(categorical_features)

preprocessor = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), categorical_features)],
    remainder='passthrough'
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.fit_transform(X_test)
test = preprocessor.fit_transform(test)
X_train

[1, 3, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


<100000x226 sparse matrix of type '<class 'numpy.float64'>'
	with 4140222 stored elements in Compressed Sparse Row format>

In [13]:
logistic_regression = LogisticRegression(C=1.0, penalty='l2', solver='lbfgs', random_state = 42)
start_time = time.time()
logistic_regression.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time)

# Making predictions on the test set
y_pred = logistic_regression.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculating precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculating recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculating F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Training time: 0.6974620819091797
Accuracy: 0.59533
Precision: 0.6016669153539531
Recall: 0.5644522657281126
F1 score: 0.5824657703855798


/Users/kseniashk/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?
__(2 балла)__

In [14]:
data = pd.read_csv('train.csv', index_col=0)
target = data['target']
data = data.drop(['target'], axis = 1)
data[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,2,2,5,1,0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
9,1,1,7,0,0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
13,5,4,9,1,0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
16,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,2,0,1,0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
19,5,1,4,0,0,0,0,0,1,0,...,4,2,0,9,0,1,0,1,1,1
20,2,1,3,1,0,0,1,0,0,0,...,3,0,0,10,0,1,0,0,1,0
22,5,1,4,0,0,1,0,0,0,0,...,7,1,3,6,1,0,1,0,1,0
26,5,1,3,1,0,0,0,1,0,0,...,4,2,1,5,0,1,0,0,0,1


In [15]:
categorical_features = []
for i in data:
    if i[-3:] == 'cat':
        categorical_features.append(i)
for feature in categorical_features:
    z = dict()
    c = dict()
    print(feature)
    for i in range(0, len(target)):
        s = data[feature].iloc[i]
        if not s in z:
            z[s] = 0
            c[s] = 0
        z[s] += target.iloc[i]
        c[s] += 1
    for i in range(0, len(target)):
        data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]

ps_ind_02_cat


/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.038139399383360445' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]


ps_ind_04_cat


/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.038643800067697166' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]


ps_ind_05_cat


...

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.033343957885645543' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]


ps_car_10_cat


/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.03646351361197196' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]


ps_car_11_cat


/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/1707723674.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.038642009384487995' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[feature].iloc[i] = z[data[feature].iloc[i]] / c[data[feature].iloc[i]]


In [16]:
data[:10]

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,2,0.038139,5,0.038644,0.033865,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
9,1,0.035725,7,0.034790,0.033865,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
13,5,0.040780,9,0.038644,0.033865,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
16,0,0.035725,2,0.034790,0.033865,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,0.038139,0,0.038644,0.033865,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
19,5,0.035725,4,0.034790,0.033865,0,0,0,1,0,...,4,2,0,9,0,1,0,1,1,1
20,2,0.035725,3,0.038644,0.033865,0,1,0,0,0,...,3,0,0,10,0,1,0,0,1,0
22,5,0.035725,4,0.034790,0.033865,1,0,0,0,0,...,7,1,3,6,1,0,1,0,1,0
26,5,0.035725,3,0.038644,0.033865,0,0,1,0,0,...,4,2,1,5,0,1,0,0,0,1


In [17]:
data2 = data

In [18]:
target = target2

In [19]:
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

In [20]:
datatypes = data.dtypes
ind = 0
for i in data:
    if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
        mn = min(X_train[i].min(), X_test[i].min())
        mx = max(X_train[i].max(), X_test[i].max())
        X_train[i] = (X_train[i] - mn) / (mx - mn)
        X_test[i] = (X_test[i] - mn) / (mx - mn)
    ind += 1

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/2632917312.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/2632917312.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/2632917312.py:4: FutureWarning: Series.__getitem__ treating keys as positions

In [21]:
logistic_regression = LogisticRegression(C=1.0, penalty='l2', solver='lbfgs', random_state = 42)
start_time = time.time()
logistic_regression.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time)

# Making predictions on the test set
y_pred = logistic_regression.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculating precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculating recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculating F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Training time: 1.7125630378723145
Accuracy: 0.58981
Precision: 0.5960827078923172
Recall: 0.5574731032276127
F1 score: 0.5761317722918583


/Users/kseniashk/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


__Вывод:__ 1 и 2 кодировка сделалии чуть-чуть лучше

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

Внесем шум:

In [38]:
data = data2
import random
for feature in categorical_features:
    for i in range(0, len(data[feature])):
        rnd = random.random() / 10
        data[feature].iloc[i] += rnd - 0.05
        #print(data[feature].iloc[i])

In [39]:
target = target2
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

In [40]:
datatypes = data.dtypes
ind = 0
for i in data:
    if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
        mn = min(X_train[i].min(), X_test[i].min())
        mx = max(X_train[i].max(), X_test[i].max())
        X_train[i] = (X_train[i] - mn) / (mx - mn)
        X_test[i] = (X_test[i] - mn) / (mx - mn)
    ind += 1

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/2632917312.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):


In [41]:
logistic_regression = LogisticRegression(C=1.0, penalty='l2', solver='lbfgs', random_state = 42)
start_time = time.time()
logistic_regression.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time)

# Making predictions on the test set
y_pred = logistic_regression.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculating precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculating recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculating F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Training time: 1.7613897323608398
Accuracy: 0.57858
Precision: 0.583741534267581
Recall: 0.5481142262928449
F1 score: 0.5653671617161715


Кросс-валидация:

In [26]:
data = data2

In [27]:
datatypes = data.dtypes
ind = 0
for i in data:
    if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
        mn = min(data[i].min(), data[i].min())
        mx = max(data[i].max(), data[i].max())
        data[i] = (data[i] - mn) / (mx - mn)
    ind += 1

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/417542964.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/417542964.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and (datatypes[ind] == 'int64' or datatypes[ind] == 'float64') and (not i in categorical_features):
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/417542964.py:4: FutureWarning: Series.__getitem__ treating keys as positions is

In [28]:
data.shape

(595212, 57)

In [29]:
target[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [31]:
data = pd.read_csv('train.csv')
target = data.target.values

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()

mse_scores = cross_val_score(regression_model, data, target, cv=5, scoring='neg_mean_squared_error')

mse_scores

array([-3.12797020e-24, -9.39238404e-28, -1.27945994e-23, -7.91049061e-28,
       -5.83026206e-27])

__Вывод:__ кросс-валидация работает очень хорошо, а шум ничего не поменял

## Часть 2. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [33]:
from sklearn.model_selection import train_test_split
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

In [34]:
datatypes = data.dtypes
ind = 0
for i in data:
    if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
        mn = min(X_train[i].min(), X_test[i].min())
        mx = max(X_train[i].max(), X_test[i].max())
        X_train[i] = (X_train[i] - mn) / (mx - mn)
        X_test[i] = (X_test[i] - mn) / (mx - mn)
    ind += 1

/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/993191153.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/993191153.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i != 'id' and datatypes[ind] == 'int64' or datatypes[ind] == 'float64':
/var/folders/vb/1jpw8q5j7wq2fm0t1yswkvt80000gn/T/ipykernel_75691/993191153.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as lab

In [35]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0)

На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [36]:
start_time = time.time()
svc.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time)

/Users/kseniashk/anaconda3/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training time: 21.4896457195282


/Users/kseniashk/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [37]:
# Making predictions on the test set
y_pred = svc.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculating precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculating recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculating F1 score
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

Accuracy: 0.72239
Precision: 0.6495904670961494
Recall: 0.9658640963084429
F1 score: 0.7767672625222138


__Вывод:__ долго(в сравнении)), но работает классно